## Nötige Software installieren

Die folgenden beiden Zellen installieren ein paar benötigte Python-Bibliotheken und laden ein paar vertrauliche Konfigurationsdaten (API-Schlüssel): 

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet chromadb  dotenv langchain langgraph langsmith langchain_chroma langchain-community langchain_openai langchain-unstructured langchain-docling unstructured-client unstructured "unstructured[pdf]" python-magic
%load_ext dotenv

In [2]:
%dotenv /home/archive/FerienkursKI/.env

In [11]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from docling.chunking import HybridChunker, HierarchicalChunker
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import chromadb
import os

In [4]:
# Pfad der PDF-Datei
pdf_path = "./data/Koalitionsvertrag-2025-1.pdf" 

chunker = HybridChunker()
loader = DoclingLoader(file_path=pdf_path, chunker=chunker)
docling = loader.load()

/opt/conda/lib/python3.11/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (2468 > 512). Running this sequence through the model will result in indexing errors


In [5]:
# Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(docling)

In [6]:
len(chunks), len(docling)

(1496, 446)

In [7]:
chunks[1].metadata['dl_meta']['doc_items'][0]['prov'][0]['page_no']

2

In [8]:
# Initialisiere Chunks und Metadaten für den VectorStore
for chunk in chunks:
    meta = {
            "source": chunk.metadata['source'],
            "heading": chunk.metadata['dl_meta']['headings'][0],
            "page": chunk.metadata['dl_meta']['doc_items'][0]['prov'][0]['page_no']
        }
    chunk.metadata = meta



In [9]:
chunks[0]

Document(metadata={'source': './data/Koalitionsvertrag-2025-1.pdf', 'heading': 'Verantwortung für Deutschland', 'page': 1}, page_content='Verantwortung für Deutschland\nKoalitionsvertrag zwischen CDU, CSU und SPD\n21. Legislaturperiode')

In [18]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

chromadb.configure(allow_reset=True)

client = chromadb.PersistentClient(path="./chroma")
#client.list_collections() 

In [26]:
client.list_collections()

['Koalitionsvertrag']

In [22]:
vector_store = Chroma(
    client=client,
    collection_name="Koalitionsvertrag",
    embedding_function=embeddings,
)

In [23]:
ids = vector_store.add_documents(documents=chunks)

In [24]:
retriever = vector_store.as_retriever()
docs = retriever.invoke("Was steht im Koalitionsvertrag zum Thema Bildung?")

In [25]:
docs

[Document(id='d20bcffd-a218-4fd7-aaeb-b1b321e9450a', metadata={'heading': '4.1. Familien, Frauen, Jugend, Senioren und Demokratie 3106', 'page': 100, 'source': './data/Koalitionsvertrag-2025-1.pdf'}, page_content='= bereits in Krippen und Kitas gelegt. Wir werden in Neubau, Ausbau, Sanierung und Modernisierung. 3126, 1 = (etwa für Inklusion, Arbeitsschutz, AusstaƩung und Digitalisierung) invesƟeren, um frühkindliche. 3127, 1 = Bildung zu ermöglichen.. 3128, 1 = Ganztag. 3129, 1 = Den Ganztagsausbau treiben wir voran. Wir halten am Ausbauziel für die Ganztagsbetreuung in der. 3130, 1 = Grundschule fest. Dafür werden wir bürokraƟsche Hürden abbauen. Der Rechtsanspruch soll. 3131, 1 = deutschlandweit mit'),
 Document(id='b0deca60-43f7-4f30-8824-d70d1357aa27', metadata={'heading': 'DemokraƟe- und Medienbildung 2339', 'page': 75, 'source': './data/Koalitionsvertrag-2025-1.pdf'}, page_content="2351, 1 = Für mehr Verlässlichkeit und Qualität im Schulsystem sowie bei der Personalgewinnung nutz